<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reviewa y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

In [1]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import json
import os
import io
import gcsfs

import pyarrow as pa
import pyarrow.parquet as pq    # Permite leer archivos en formato parquet
import glob                     # crear listas de archivos a partir de búsquedas con comodines en un directorio

- - -
### Trabajando con el dataset de 

In [3]:
client = bigquery.Client()
sql = """
    SELECT gmre.user_id, gmre.name, gmre.time, gmre.rating, 
           gmre.text, gmre.gmap_id, gmre.Estado, gmre.state
    FROM `eternal-empire-399016.gmy_bq.df_gm_reviews_estados` AS gmre;
      """
df_gm_reviews_estados = client.query(sql).to_dataframe()
df_gm_reviews_estados.head(5)

,user_id,name,time,rating,text,gmap_id,Estado,state
0,105888220566790421746,Susan Munyan,1537898147002,5,Most of the employees are friendly and helpful...,0x884732fc89b402e9:0x338840be58747f91,Ohio,OH
1,109304393660194561517,Mad C,1481608818681,5,Super friendly and chill staff. A great locati...,0x88470f9bba97d377:0xae741cb72ffb4ac6,Ohio,OH
2,115370949688658940178,D Wilson,1522109428633,5,My first GO-TO when working on my home... alwa...,0x8830e2cfda2c28f1:0xa1f75dac5da4807d,Ohio,OH
3,109415200712115003587,ro barb,1583616397667,5,"The only Italian/Sicilian Church in Akron, tha...",0x883128262fbcb01d:0xb5d28ab2db9110a5,Ohio,OH
4,115670067159974015279,Leslie Carter,1515365462273,5,Nice club,0x88408595137955b3:0x4f7e732643723d0,Ohio,OH


In [4]:
client = bigquery.Client()
sql = """
    SELECT *
    FROM `gmy_bq.a_vista_gm_reviews_estados_and_gm_md_sitios_filtrada`;
      """
df_gm_re_ms_sitios = client.query(sql).to_dataframe()
df_gm_re_ms_sitios.head(5)

BadRequest: 400 Resources exceeded during query execution: Your project or organization exceeded the maximum disk and memory limit available for shuffle operations. Consider provisioning more slots, reducing query concurrency, or using more efficient logic in this job.

Location: us-east1
Job ID: 92cc9649-6e72-4c0f-a451-19d2733c00b2


In [4]:
df0.head(10).T

,0,1,2,3,4,5,6,7,8,9
review_id,KU_O5udG6zpxOg-VcAEodg,lUUhg8ltDsUZ9h0xnwY4Dg,JBWZmBy69VMggxj3eYn17Q,E9AB7V4z8xrt2uPF7T55FQ,A4n4YaE-owOVgTQcrVqHUw,4KpIldEM-tdnrJLqYzRfZQ,meGaFP7yxQdjyABrYDVeoQ,tXHWJWnTdrraHGUqaPWj3g,r1tPwFMILy0COeEQ-B3YLw,zcj7iTXdSz0GCBOReMTeiQ
user_id,mh_-eMZ6K5RLWhZyISBhwA,RreNy--tOmXMl1en0wiBOg,aFa96pz67TwOFu4Weq5Agg,iYY5Ii1LGpZCpXFkHlMefw,S7bjj-L07JuRr-tpX1UZLw,Z5j9Xw_G0c7M2b1-iS67wg,_jaJDV-qTBafatbObmtzpA,zKAHSNzqvwsyoFCw3QpafA,3M1_pyDSgMP6sRMz564eJw,X8XCFMZN8pFlWEZcKuKzZw
business_id,XQfwVwDr-v0ZS3_CbbE5Xw,cPepkJeRMtHapc_b2Oe_dw,kq5Ghhh14r-eCxlVmlyd8w,Zx7n8mdt8OzLRXVzolXNhQ,I6L0Zxi5Ww0zEWSAVgngeQ,HTqXI5S2XcSlh_ylx9sE6g,cg4JFJcCxRTTMmcg9O9KtA,S2Ho8yLxhKAa26pBAm6rxA,8xM8akbQhHDQdJO1sPMB1A,Zx7n8mdt8OzLRXVzolXNhQ
stars,3,4,5,5,4,5,1,4,5,5
useful,0,1,0,0,0,1,0,0,0,0
funny,0,0,0,0,0,1,1,0,0,0
cool,0,1,0,0,0,1,0,0,0,0
text,"If you decide to eat here, just be aware it is...",I was really between 3 and 4 stars for this on...,My boyfriend and I tried this deli for the fir...,Amazing biscuits and (fill in the blank). Grea...,The cafe was extremely cute. We came at 8am an...,I've only had the cannolis here but they are a...,Skip this train wreck if you are looking for d...,Enjoyed my fish out at a sidewalk table. A bi...,I had the pleasure to meet with Ann today and ...,A menu that satisfies everyone's cravings! Cle...
date,2018-07-07,2018-07-17,2018-08-23,2018-04-27,2018-07-07,2018-03-23,2018-02-11,2018-05-19,2018-07-26,2018-01-21


- - -
### Cargando todos los archivos parquet del dataset de review como un Dataframe

In [7]:
# Obtener el proyecto actual
project_id = os.environ.get("Proyecto Final - Henry")

# Crear un cliente de Cloud Storage
client = storage.Client(project=project_id)

# Obtener el nombre del bucket
bucket_name = "gmy"

# Obtener el objeto de referencia al bucket
bucket = client.bucket(bucket_name)

# Obtener una lista de las carpetas de los estados
folders = [x for x in bucket.list_blobs(prefix="dataset/review/")]

# Crear un DataFrame vacío
df = pd.DataFrame()

# Iterar sobre la carpeta de review
for folder in folders:
    archivo = "gs://gmy/" + folder.name
    print("Procesando archivo:... ", archivo)

    # Leer el archivo Parquet en un DataFrame
    df_review = pd.read_parquet(archivo, storage_options={"project": "Proyecto Final - Henry"})
    
    # Se le quita la columna funny
    df_review =  df_review.drop('funny', axis=1)

    # Agregar los datos al DataFrame
    df = pd.concat([df, df_review], ignore_index=True)    
    
# Guardar el DataFrame en formato csv
ruta_archivos_ETL = "gs://gmy/etl/"
nombre_archivo_csv = "yelp_reviews.csv"
nombre_archivo_parquet = "yelp_reviews.parquet"
#print("Creando el archivo: ", nombre_archivo_csv, "en: ", ruta_archivos_ETL)
#df.to_csv(ruta_archivos_ETL + nombre_archivo_csv, index=False)
print("Creando el archivo: ", nombre_archivo_parquet, "en: ", ruta_archivos_ETL)
df.to_parquet(ruta_archivos_ETL + nombre_archivo_parquet)
print("Archivo creado")

Procesando archivo:...  gs://gmy/dataset/review/review-reducido_0.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_1.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_10.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_11.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_12.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_13.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_14.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_15.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_16.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_17.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_18.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_19.parquet
Procesando archivo:...  gs://gmy/dataset/review/review-reducido_2.parquet
Procesando archivo:...  gs:/

In [45]:
df.head().T

,0,1,2,3,4
review_id,KU_O5udG6zpxOg-VcAEodg,lUUhg8ltDsUZ9h0xnwY4Dg,JBWZmBy69VMggxj3eYn17Q,E9AB7V4z8xrt2uPF7T55FQ,A4n4YaE-owOVgTQcrVqHUw
user_id,mh_-eMZ6K5RLWhZyISBhwA,RreNy--tOmXMl1en0wiBOg,aFa96pz67TwOFu4Weq5Agg,iYY5Ii1LGpZCpXFkHlMefw,S7bjj-L07JuRr-tpX1UZLw
business_id,XQfwVwDr-v0ZS3_CbbE5Xw,cPepkJeRMtHapc_b2Oe_dw,kq5Ghhh14r-eCxlVmlyd8w,Zx7n8mdt8OzLRXVzolXNhQ,I6L0Zxi5Ww0zEWSAVgngeQ
stars,3,4,5,5,4
useful,0,1,0,0,0
cool,0,1,0,0,0
text,"If you decide to eat here, just be aware it is...",I was really between 3 and 4 stars for this on...,My boyfriend and I tried this deli for the fir...,Amazing biscuits and (fill in the blank). Grea...,The cafe was extremely cute. We came at 8am an...
date,2018-07-07,2018-07-17,2018-08-23,2018-04-27,2018-07-07


In [50]:
# Revisando el archivo yelp_review de tipo csv
ruta_archivos_ETL = "gs://gmy/etl/yelp_reviews.csv"
df = pd.read_csv(ruta_archivos_ETL)


In [11]:
df.head().T

,0,1,2,3,4
review_id,KU_O5udG6zpxOg-VcAEodg,lUUhg8ltDsUZ9h0xnwY4Dg,JBWZmBy69VMggxj3eYn17Q,E9AB7V4z8xrt2uPF7T55FQ,A4n4YaE-owOVgTQcrVqHUw
user_id,mh_-eMZ6K5RLWhZyISBhwA,RreNy--tOmXMl1en0wiBOg,aFa96pz67TwOFu4Weq5Agg,iYY5Ii1LGpZCpXFkHlMefw,S7bjj-L07JuRr-tpX1UZLw
business_id,XQfwVwDr-v0ZS3_CbbE5Xw,cPepkJeRMtHapc_b2Oe_dw,kq5Ghhh14r-eCxlVmlyd8w,Zx7n8mdt8OzLRXVzolXNhQ,I6L0Zxi5Ww0zEWSAVgngeQ
stars,3,4,5,5,4
useful,0,1,0,0,0
cool,0,1,0,0,0
text,"If you decide to eat here, just be aware it is...",I was really between 3 and 4 stars for this on...,My boyfriend and I tried this deli for the fir...,Amazing biscuits and (fill in the blank). Grea...,The cafe was extremely cute. We came at 8am an...
date,2018-07-07,2018-07-17,2018-08-23,2018-04-27,2018-07-07


In [7]:
### Probando leyendo el archivo Parquet del dataset reviews
# Ruta completa en Cloud Storage
archivo = "gs://gmy/eda/yelp_reviews.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df_yelp_reviews_5_annos = pd.read_parquet(archivo, storage_options={"project": "Proyecto Final - Henry"})
df_yelp_reviews_5_annos.shape

(2742210, 8)

In [8]:
df_yelp_reviews_5_annos.head().T

,0,1,2,3,4
review_id,KU_O5udG6zpxOg-VcAEodg,lUUhg8ltDsUZ9h0xnwY4Dg,JBWZmBy69VMggxj3eYn17Q,E9AB7V4z8xrt2uPF7T55FQ,A4n4YaE-owOVgTQcrVqHUw
user_id,mh_-eMZ6K5RLWhZyISBhwA,RreNy--tOmXMl1en0wiBOg,aFa96pz67TwOFu4Weq5Agg,iYY5Ii1LGpZCpXFkHlMefw,S7bjj-L07JuRr-tpX1UZLw
business_id,XQfwVwDr-v0ZS3_CbbE5Xw,cPepkJeRMtHapc_b2Oe_dw,kq5Ghhh14r-eCxlVmlyd8w,Zx7n8mdt8OzLRXVzolXNhQ,I6L0Zxi5Ww0zEWSAVgngeQ
stars,3,4,5,5,4
useful,0,1,0,0,0
cool,0,1,0,0,0
text,"If you decide to eat here, just be aware it is...",I was really between 3 and 4 stars for this on...,My boyfriend and I tried this deli for the fir...,Amazing biscuits and (fill in the blank). Grea...,The cafe was extremely cute. We came at 8am an...
date,2018-07-07,2018-07-17,2018-08-23,2018-04-27,2018-07-07


In [9]:
### Separando el dataset yelp_reviews en 2 grupos por años:
### Primer grupo 3 años de datos: desde el 2018 al 2020
### Segundo grupo 2 años de datos: desde el 2021 al 2022

In [28]:
### Probando leyendo el archivo Parquet del dataset reviews
# Ruta completa en Cloud Storage
archivo = "gs://gmy/eda/yelp_reviews.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df_yelp_reviews_5_annos = pd.read_parquet(archivo, storage_options={"project": "Proyecto Final - Henry"})
df_yelp_reviews_5_annos.shape

# Separando en grupos por año
print('Datos iniciales desde: ', df_yelp_reviews_5_annos['date'].min(), 'hasta: ',df_yelp_reviews_5_annos['date'].max())
df_review_2018_to_2020 = df_yelp_reviews_5_annos[df_yelp_reviews_5_annos['date'] <= '2020-12-31']
df_review_2018_to_2020 = df_review_2018_to_2020.reset_index(drop=True)
df_review_2021_to_2022 = df_yelp_reviews_5_annos[df_yelp_reviews_5_annos['date'] > '2020-12-31']
df_review_2021_to_2022 = df_review_2021_to_2022.reset_index(drop=True)

# Guardar el DataFrame en formato csv
ruta_archivos_EDA = "gs://gmy/eda/"

nombre_archivo_parquet_1er = "yelp_reviews_2018_to_2020.parquet"
print('Datos 1er grupo 3 años desde: ', df_review_2018_to_2020['date'].min(), 'hasta: ',df_review_2018_to_2020['date'].max())
print("Creando el archivo: ", nombre_archivo_parquet_1er, "en: ", ruta_archivos_EDA)
df_review_2018_to_2020.to_parquet(ruta_archivos_EDA + nombre_archivo_parquet_1er)

nombre_archivo_parquet_2do = "yelp_reviews_2021_to_2022.parquet"
print('Datos 2do grupo 2 años desde: ', df_review_2021_to_2022['date'].min(), 'hasta: ',df_review_2021_to_2022['date'].max())
print("Creando el archivo: ", nombre_archivo_parquet_2do, "en: ", ruta_archivos_EDA)
df_review_2021_to_2022.to_parquet(ruta_archivos_EDA + nombre_archivo_parquet_2do)

print("Archivos creados!")

Datos iniciales desde:  2018-01-01 hasta:  2022-01-19
Datos 1er grupo 3 años desde:  2018-01-01 hasta:  2020-12-31
Creando el archivo:  yelp_reviews_2018_to_2020.parquet en:  gs://gmy/eda/
Datos 2do grupo 2 años desde:  2021-01-01 hasta:  2022-01-19
Creando el archivo:  yelp_reviews_2021_to_2022.parquet en:  gs://gmy/eda/
Archivos creados!


In [29]:
### Probando leyendo el archivo Parquet del dataset reviews
# Ruta completa en Cloud Storage
archivo = "gs://gmy/eda/yelp_reviews.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df_yelp_reviews_5_annos = pd.read_parquet(archivo, storage_options={"project": "Proyecto Final - Henry"})
df_yelp_reviews_5_annos.shape

# Separando en grupos por año
print('Datos iniciales desde: ', df_yelp_reviews_5_annos['date'].min(), 'hasta: ',df_yelp_reviews_5_annos['date'].max())
df_review_2018 = df_yelp_reviews_5_annos[df_yelp_reviews_5_annos['date'].between('2018-01-01','2018-12-31')]
df_review_2018 = df_review_2018.reset_index(drop=True)
df_review_2019 = df_yelp_reviews_5_annos[df_yelp_reviews_5_annos['date'].between('2019-01-01','2019-12-31')]
df_review_2019 = df_review_2019.reset_index(drop=True)
df_review_2020 = df_yelp_reviews_5_annos[df_yelp_reviews_5_annos['date'].between('2020-01-01','2020-12-31')]
df_review_2020 = df_review_2020.reset_index(drop=True)
df_review_2021 = df_yelp_reviews_5_annos[df_yelp_reviews_5_annos['date'].between('2021-01-01','2021-12-31')]
df_review_2021 = df_review_2021.reset_index(drop=True)
df_review_2022 = df_yelp_reviews_5_annos[df_yelp_reviews_5_annos['date'].between('2022-01-01','2022-12-31')]
df_review_2022 = df_review_2022.reset_index(drop=True)

# Guardar el DataFrame en formato csv
ruta_archivos_EDA = "gs://gmy/eda/"

nombre_archivo_parquet_2018 = "yelp_review_2018.parquet"
print('Datos 2018: ', df_review_2018['date'].min(), 'hasta: ',df_review_2018['date'].max())
print("Creando el archivo: ", nombre_archivo_parquet_2018, "en: ", ruta_archivos_EDA)
df_review_2018.to_parquet(ruta_archivos_EDA + nombre_archivo_parquet_2018)

nombre_archivo_parquet_2019 = "yelp_review_2019.parquet"
print('Datos 2019: ', df_review_2019['date'].min(), 'hasta: ',df_review_2019['date'].max())
print("Creando el archivo: ", nombre_archivo_parquet_2019, "en: ", ruta_archivos_EDA)
df_review_2019.to_parquet(ruta_archivos_EDA + nombre_archivo_parquet_2019)

nombre_archivo_parquet_2020 = "yelp_review_2020.parquet"
print('Datos 2020: ', df_review_2020['date'].min(), 'hasta: ',df_review_2020['date'].max())
print("Creando el archivo: ", nombre_archivo_parquet_2020, "en: ", ruta_archivos_EDA)
df_review_2020.to_parquet(ruta_archivos_EDA + nombre_archivo_parquet_2020)

nombre_archivo_parquet_2021 = "yelp_review_2021.parquet"
print('Datos 2021: ', df_review_2021['date'].min(), 'hasta: ',df_review_2021['date'].max())
print("Creando el archivo: ", nombre_archivo_parquet_2021, "en: ", ruta_archivos_EDA)
df_review_2021.to_parquet(ruta_archivos_EDA + nombre_archivo_parquet_2021)

nombre_archivo_parquet_2022 = "yelp_review_2022.parquet"
print('Datos 2022: ', df_review_2022['date'].min(), 'hasta: ',df_review_2022['date'].max())
print("Creando el archivo: ", nombre_archivo_parquet_2022, "en: ", ruta_archivos_EDA)
df_review_2022.to_parquet(ruta_archivos_EDA + nombre_archivo_parquet_2022)

print("Archivos creados!")

Datos iniciales desde:  2018-01-01 hasta:  2022-01-19
Datos 2018:  2018-01-01 hasta:  2018-12-31
Creando el archivo:  yelp_review_2018.parquet en:  gs://gmy/eda/
Datos 2019:  2019-01-01 hasta:  2019-12-31
Creando el archivo:  yelp_review_2019.parquet en:  gs://gmy/eda/
Datos 2020:  2020-01-01 hasta:  2020-12-31
Creando el archivo:  yelp_review_2020.parquet en:  gs://gmy/eda/
Datos 2021:  2021-01-01 hasta:  2021-12-31
Creando el archivo:  yelp_review_2021.parquet en:  gs://gmy/eda/
Datos 2022:  2022-01-01 hasta:  2022-01-19
Creando el archivo:  yelp_review_2022.parquet en:  gs://gmy/eda/
Archivos creados!


In [27]:
### Probando leyendo el archivo Parquet del dataset reviews
# Ruta completa en Cloud Storage
archivo = "gs://gmy/eda/yelp_review_2019.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df2 = pd.read_parquet(archivo, storage_options={"project": "Proyecto Final - Henry"})
df2.head().T

,24225,24226,24229,25298,25299
review_id,F6VdYuJiefNBfn3HNELv0A,nAMDCKElSKxOhzm9Lpt6Eg,3CmdoGKBZUX3Nb5IfbztMg,-4Nv_JAolCM0gzKM4DZpmQ,_eQuEUgJkgHQjKEO00ZK9Q
user_id,s4sR0rvVOf6iby77xGeyLg,SRp90x9d2719GOZ_PT-a6A,IDOQnhI1GOkLIDaTbBxkfw,NjuPidqqsNW9KG8PBPrycg,X71-eKfjEgfSxApe2MtrfQ
business_id,z7em5co2qckbAXoDGXynsA,M0r9lUn2gLFYgIwIfG8-bQ,itAhmbhHOyQQparfwicjDQ,-3AooxIkg38UyUdlz5oXdw,TVGuOv0Nc0omLXtGKVOwdQ
stars,5,5,5,4,1
useful,0,0,0,2,0
cool,0,0,0,1,0
text,The food is INCREDIBLE! We didn't have time to...,"We had a great time, and excellent service. Al...",My favorite coffee shop in New Orleans for sur...,"Old school circa 1979, the cozy, intimate banq...","All Lowes in 20 mile radius, stopped\nstocking..."
date,2019-01-04,2019-01-06,2019-01-27,2019-02-17,2019-02-17


In [4]:
### Probando leyendo el archivo Parquet del dataset reviews
# Ruta completa en Cloud Storage
archivo = "gs://gmy/eda/yelp_reviews_2021_to_2022.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df3 = pd.read_parquet(archivo, storage_options={"project": "Proyecto Final - Henry"})
df3.head().T

,58027,59256,59984,60376,61890
review_id,iBUJvIOkToh2ZECVNq5PDg,HgEofz6qEQqKYPT7YLA34w,milJ7UH4Od9pBe2gWac9tA,Kxo5d6EOnOE-vERwQf2a1w,STqHwh6xd05bgS6FoAgRqw
user_id,iAD32p6h32eKDVxsPHSRHA,rYvWv-Ny16b1lMcw1IP7JQ,v7i4M7NIx3bMNMChaXjU7Q,2ntnbUia9Bna62W0fqNcxg,j4qNLF-VNRF2DwBkUENW-w
business_id,YB26JvvGS2LgkxEKOObSAw,jfIwOEXcVRyhZjM4ISOh4g,raKflkp3CANr8N7qpQ3ZyQ,S-VD26LE_LeJNx5nASk_pw,yE1raqkLX7OZsjmX3qKIKg
stars,5,1,5,5,5
useful,0,0,0,0,0
cool,0,0,0,0,0
text,I've been eating at this restaurant for over 5...,How does a delivery person from here get lost ...,I WISH I was still a Sierra resident. They're ...,"The service is always good, the employees are ...",two words: whipped. feta. \nexplosion of amazi...
date,2021-01-08,2021-01-02,2021-02-02,2021-01-26,2021-01-27


- - -
### Trabajando con el dataset de reviews-estados

lista_estados = ['Tennessee', 'Ohio', 'North Carolina', 'Georgia', 'Texas',
                 'Michigan', 'Alabama', 'Indiana', 'Arizona', 'Florida']

lista_iniciales_estados = ['TN', 'OH', 'NC', 'GA', 'TX',
                 'MI', 'AL', 'IN', 'AZ', 'FL']

| Estado | (Abreviatura) | Capital |
| ------ | ------------- | ------- |
| Alabama | (AL) | Montgomery |<br>
| Arizona | (AZ) | Phoenix |<br>
| Carolina del Norte | (NC) | Raleigh |<br>
| Florida | (FL) | Tallahassee |<br>
| Georgia | (GA) | Atlanta |<br>
| Indiana | (IN) | Indianápolis |<br>
| Míchigan | (MI) | Lansing |<br>
| Ohio | (OH) | Columbus |<br>
| Tennessee | (TN) | Nashville |<br>
| Texas | (TX) | Austin |<br>                

dicc_estados_iniciales = {'Tennessee':'TN', 'Ohio':'OH', 'North Carolina':'NC', 'Georgia':'GA', 'Texas':'TX', 'Michigan':'MI', 'Alabama':'AL', 'Indiana':'IN', 'Arizona':'AZ', 'Florida':'FL'}

dicc_estados_iniciales = {'Michigan':'MI', 'Alabama':'AL', 'Indiana':'IN', 'Arizona':'AZ', 'Florida':'FL'}

dicc_estados_iniciales = {'Tennessee':'TN', 'Ohio':'OH', 'North Carolina':'NC', 'Georgia':'GA', 'Texas':'TX'}

In [53]:
### Probando leyendo el primer archivo Parquet del dataset reviews-estados
# Ruta completa en Cloud Storage
archivo = "gs://gmy/dataset/Datasets_Google_Maps/reviews-estados/review-Florida/review-Florida_1.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df0 = pd.read_parquet(archivo, storage_options={"project": "Proyecto Final - Henry"})
df0.head().T

,0,1,2,3,4
user_id,101471856155148729010,115477234789038326051,101805010244892834381,106344422881493743981,100875113069561776529
name,Julie A. Gerber,Martin Sheffield,Brian Truett,Tina Sun,James Haynes
time,1628003250740,1595031217005,1522924253567,1467907819586,1480683415081
rating,1,5,5,1,5
text,Update: Their “reply” to my review amounted to...,He's a knowledgeable doctor but the way he run...,"Best doctor I've ever had, I never wait to be ...",I was told he is a good doctor. I was trying t...,Takes the time to actually get to know his pat...
pics,None,None,None,None,None
resp,{'text': 'Thank you for the feedback. We would...,{'text': 'Thank you for reaching out. I’m forw...,None,None,None
gmap_id,0x8893863ea87bd5dd:0x9383ebf973e74abb,0x8893863ea87bd5dd:0x9383ebf973e74abb,0x8893863ea87bd5dd:0x9383ebf973e74abb,0x8893863ea87bd5dd:0x9383ebf973e74abb,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [6]:
lista_inicial_estados = ['Michigan', 'Alabama', 'Indiana', 'Arizona', 'Florida']
dicc_estados_iniciales = {'Michigan':'MI', 'Alabama':'AL', 'Indiana':'IN', 'Arizona':'AZ', 'Florida':'FL'}

# Obtener el proyecto actual
project_id = os.environ.get("Proyecto Final - Henry")

# Crear un cliente de Cloud Storage
client = storage.Client(project=project_id)

# Obtener el nombre del bucket
bucket_name = "gmy"

# Obtener el objeto de referencia al bucket
bucket = client.bucket(bucket_name)

# Obtener una lista de las carpetas de los estados
folders = [x for x in bucket.list_blobs(prefix="dataset/Datasets_Google_Maps/reviews-estados/")]

# Crear un DataFrame vacío
df = pd.DataFrame()

# Iterar sobre la carpeta de review
for folder in folders:
    # Obtener el nombre del estado
    archivo = folder.name.split("/")[-1]  
    estado = (archivo.split("-")[1]).split("_")[0]
    # ruta completa del archivo parquet
    archivo_parquet = "gs://gmy/" + folder.name
    # Se procesa una lista inicial de 5 estados
    # 'Michigan', 'Alabama', 'Indiana', 'Arizona' y 'Florida'
    if estado in lista_inicial_estados:
        print("Procesando... ", archivo)

        # Leer el archivo Parquet en un DataFrame
        df_review = pd.read_parquet(archivo_parquet, storage_options={"project": "Proyecto Final - Henry"})

        # Agregar la columna Estado
        df_review["Estado"] = estado
        df_review["state"] = dicc_estados_iniciales[estado]

        # Agregar los datos al DataFrame
        df = pd.concat([df, df_review], ignore_index=True)     

# Guardar el DataFrame en formato csv
ruta_archivos_ETL = "gs://gmy/etl/"
#nombre_archivo_csv = "reviews-estados.csv"
nombre_archivo_parquet = "reviews-estados_iniciales.parquet"
##print("Creando el archivo: ", nombre_archivo_csv, "en: ", ruta_archivos_ETL)
##df.to_csv(ruta_archivos_ETL + nombre_archivo_csv, index=False, escapechar="\\")
print("Creando el archivo: ", nombre_archivo_parquet, "en: ", ruta_archivos_ETL)
df.to_parquet(ruta_archivos_ETL + nombre_archivo_parquet)
print("Archivo creado")

Procesando...  review-Alabama_1.parquet
Procesando...  review-Alabama_10.parquet
Procesando...  review-Alabama_11.parquet
Procesando...  review-Alabama_12.parquet
Procesando...  review-Alabama_2.parquet
Procesando...  review-Alabama_3.parquet
Procesando...  review-Alabama_4.parquet
Procesando...  review-Alabama_5.parquet
Procesando...  review-Alabama_6.parquet
Procesando...  review-Alabama_7.parquet
Procesando...  review-Alabama_8.parquet
Procesando...  review-Alabama_9.parquet
Procesando...  review-Arizona_1.parquet
Procesando...  review-Arizona_10.parquet
Procesando...  review-Arizona_11.parquet
Procesando...  review-Arizona_12.parquet
Procesando...  review-Arizona_13.parquet
Procesando...  review-Arizona_14.parquet
Procesando...  review-Arizona_2.parquet
Procesando...  review-Arizona_3.parquet
Procesando...  review-Arizona_4.parquet
Procesando...  review-Arizona_5.parquet
Procesando...  review-Arizona_6.parquet
Procesando...  review-Arizona_7.parquet
Procesando...  review-Arizona_8.

In [5]:
# Revisando el archivo review-estados de tipo csv
df.tail().T

,11249995,11249996,11249997,11249998,11249999
user_id,112155099300432389570,105517427925758780495,102173065846910103712,114643684551556785044,116512240055207547540
name,Larry Stehlik,George Styka,Edward Seiler,Hector MrHEME Espinola,David Gruesbeck
time,1499555836404,1488512224300,1557579251594,1554895749450,1568258894157
rating,5,4,5,5,5
text,I only live 2 blocks away. Fellow parishioners...,This is a good Catholic Church first small tow...,The only reason I was there was because they w...,Excellent place for Spiritual Nourishment,Very. Ice Church
pics,None,None,None,None,None
resp,None,None,None,None,None
gmap_id,0x882241e5afaf1e11:0x5934c1dae78d564f,0x882241e5afaf1e11:0x5934c1dae78d564f,0x882241e5afaf1e11:0x5934c1dae78d564f,0x882241e5afaf1e11:0x5934c1dae78d564f,0x882241e5afaf1e11:0x5934c1dae78d564f
Estado,Michigan,Michigan,Michigan,Michigan,Michigan
state,MI,MI,MI,MI,MI


In [ ]:
# Revisando el archivo review-estados de tipo csv
ruta_archivos_ETL = "gs://gmy/etl/reviews-estados.csv"
df0 = pd.read_csv(ruta_archivos_ETL)
print(df0.tail())
print(df0.shape)

In [2]:
### Probando leyendo el archivo completo Parquet del dataset reviews-estados
# Ruta completa en Cloud Storage
archivo = "gs://gmy/etl/reviews-estados.parquet"

# Lee el archivo Parquet directamente en un DataFrame de pandas
df0 = pd.read_parquet(archivo, storage_options={"project": "Proyecto Final - Henry"})
df0.head().T

,0,1,2,3,4
user_id,114043824230907811356,116009008420407071921,106239928178034609776,104970056115801744726,110587471700525107383
name,Kanisha Mixon,Brandie Hodges,Sharon King,Veronica Pierce,Whitney Waldon Collier
time,1597168272670,1609899039594,1547235290843,1517709403534,1535245718492
rating,5,5,4,5,5
text,Very Personable staff! Beautiful and clean env...,Best clothing intown,None,None,None
pics,None,None,None,None,None
resp,None,None,None,None,None
gmap_id,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
Estado,Alabama,Alabama,Alabama,Alabama,Alabama


In [6]:
df0.shape

(11250000, 9)

- - -
Revisando el dataset tip

In [3]:
# Lee el archivo Parquet directamente en un DataFrame de pandas
ruta_archivos_ETL = "gs://gmy/eda/yelp_tip.parquet"
df0 = pd.read_parquet(ruta_archivos_ETL, storage_options={"project": "Proyecto Final - Henry"})
df0.head().T

,0,1,2,3,4
user_id,1viszljzsa8W7VnYPuWosw,YnlCpuaBa3qWBp4te8pGmA,XJDLaoN1PerKw2woiKeepA,9M_nk0kA3CTJt6UoxRAK_g,0LY6l9-Z2IrVtnfj6OX06w
business_id,Zg83olSjsdXAhE5EEy5IcQ,XIKYdKWq72zUYsq8NBxcCQ,t0zwddmbOGQOADrAxEPHQQ,oJf3IZloLf_PDeEeZW5nDg,Zmwm6d872C8kWJr4b6UGfg
text,Order crispy and they bake it up good.,The honey glazed salmon is amazing!,Come by for some Jelly Bean sours this weekend!,Meatball sandwich,Great tasty food
date,2018-04-01 00:00:00,2018-01-14 00:00:00,2018-03-31 00:00:00,2018-03-27 00:00:00,2018-04-01 00:00:00
compliment_count,0,0,0,0,0


In [4]:
df0.shape

(193030, 5)

In [20]:
df0.columns

Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')

- - -
### Trabajando con el dataset de gm_metadata_sitios_misc
### Se tiene una columna con el nombre descripción (con acento) falla al momento de cargar el dataset

In [2]:
# Lee el archivo Parquet directamente en un DataFrame de pandas
ruta_archivos_ETL = "gs://gmy/eda/metadata-sitios.parquet"
df0 = pd.read_parquet(ruta_archivos_ETL, storage_options={"project": "Proyecto Final - Henry"})
df0.head().T

,0,1,2,3,4
gmap_id,0x88f16e41928ff687:0x883dad4fd048e8f8,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,0x80c2c778e3b73d33:0xbdc58662a4a97d49,0x80c2c89923b27a41:0x32041559418d447,0x80c2c632f933b073:0xc31785961fe826a6
name,Porter Pharmacy,City Textile,San Soo Dang,Nova Fabrics,Nobel Textile Co
address,"Porter Pharmacy, 129 N Second St, Cochran, GA ...","City Textile, 3001 E Pico Blvd, Los Angeles, C...","San Soo Dang, 761 S Vermont Ave, Los Angeles, ...","Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...","Nobel Textile Co, 719 E 9th St, Los Angeles, C..."
latitude,32.3883,34.018891,34.058092,34.023669,34.036694
longitude,-83.3571,-118.21529,-118.29213,-118.23293,-118.249421
avg_rating,4.9,4.5,4.4,3.3,4.3
num_of_reviews,16,6,18,6,7
state,Open ⋅ Closes 6PM,Open now,Open ⋅ Closes 6PM,Open ⋅ Closes 5PM,Open ⋅ Closes 5PM


In [3]:
# Lee el archivo Parquet directamente en un DataFrame de pandas
ruta_archivos_ETL = "gs://gmy/eda/metadata-sitios-MISC-servicios.parquet"
df1 = pd.read_parquet(ruta_archivos_ETL, storage_options={"project": "Proyecto Final - Henry"})
df1.head().T

,0,1,2,3,4
gmapid,0x88f16e41928ff687:0x883dad4fd048e8f8,0x88f16e41928ff687:0x883dad4fd048e8f8,0x88f16e41928ff687:0x883dad4fd048e8f8,0x88f16e41928ff687:0x883dad4fd048e8f8,0x88f16e41928ff687:0x883dad4fd048e8f8
servicio,Accessibility,Health & safety,Health & safety,Planning,Service options
descripción,Wheelchair accessible entrance,Mask required,Staff required to disinfect surfaces between v...,Quick visit,In-store shopping


In [4]:
# Lee el archivo Parquet directamente en un DataFrame de pandas
ruta_archivos_ETL = "gs://gmy/eda/eda_metadata_sitios.parquet"
df1 = pd.read_parquet(ruta_archivos_ETL, storage_options={"project": "Proyecto Final - Henry"})
df1.head().T

,0,1,2,3,4
gmap_id,0x88f16e41928ff687:0x883dad4fd048e8f8,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,0x80c2c778e3b73d33:0xbdc58662a4a97d49,0x80c2c89923b27a41:0x32041559418d447,0x80c2c632f933b073:0xc31785961fe826a6
address,"Porter Pharmacy, 129 N Second St, Cochran, GA ...","City Textile, 3001 E Pico Blvd, Los Angeles, C...","San Soo Dang, 761 S Vermont Ave, Los Angeles, ...","Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...","Nobel Textile Co, 719 E 9th St, Los Angeles, C..."
latitude,32.3883,34.018891,34.058092,34.023669,34.036694
longitude,-83.3571,-118.21529,-118.29213,-118.23293,-118.249421
avg_rating,4.9,4.5,4.4,3.3,4.3
num_of_reviews,16,6,18,6,7
state,Open ⋅ Closes 6PM,Open now,Open ⋅ Closes 6PM,Open ⋅ Closes 5PM,Open ⋅ Closes 5PM


- - -

In [ ]:
lista_inicial_estados = ['Michigan', 'Alabama', 'Indiana', 'Arizona', 'Florida','Tennessee', 'Ohio', 'Georgia', 'Texas']
dicc_estados_iniciales = {'Michigan':'MI', 'Alabama':'AL', 'Indiana':'IN', 'Arizona':'AZ', 'Florida':'FL' 'Tennessee':'TN', 'Ohio':'OH', 'Georgia':'GA', 'Texas':'TX'}

# Obtener el proyecto actual
project_id = os.environ.get("Proyecto Final - Henry")

# Crear un cliente de Cloud Storage
client = storage.Client(project=project_id)

# Obtener el nombre del bucket
bucket_name = "gmy"

# Obtener el objeto de referencia al bucket
bucket = client.bucket(bucket_name)

# Obtener una lista de las carpetas de los estados
folders = [x for x in bucket.list_blobs(prefix="dataset/Datasets_Google_Maps/reviews-estados/")]

# Crear un DataFrame vacío
df = pd.DataFrame()

# Iterar sobre la carpeta de review
for folder in folders:
    # Obtener el nombre del estado
    archivo = folder.name.split("/")[-1]  
    estado = (archivo.split("-")[1]).split("_")[0]
    # ruta completa del archivo parquet
    archivo_parquet = "gs://gmy/" + folder.name
    # Se procesa una lista inicial de 5 estados
    # 'Michigan', 'Alabama', 'Indiana', 'Arizona' y 'Florida'
    if estado in lista_inicial_estados:
        print("Procesando... ", archivo)

        # Leer el archivo Parquet en un DataFrame
        df_review = pd.read_parquet(archivo_parquet, storage_options={"project": "Proyecto Final - Henry"})

        # Agregar la columna Estado
        df_review["Estado"] = estado
        df_review["state"] = dicc_estados_iniciales[estado]

        # Agregar los datos al DataFrame
        df = pd.concat([df, df_review], ignore_index=True)     

# Guardar el DataFrame en formato csv
ruta_archivos_ETL = "gs://gmy/eda/"
#nombre_archivo_csv = "reviews-estados.csv"
nombre_archivo_parquet = "reviews-estados_todos.parquet"
##print("Creando el archivo: ", nombre_archivo_csv, "en: ", ruta_archivos_ETL)
##df.to_csv(ruta_archivos_ETL + nombre_archivo_csv, index=False, escapechar="\\")
print("Creando el archivo: ", nombre_archivo_parquet, "en: ", ruta_archivos_ETL)
df.to_parquet(ruta_archivos_ETL + nombre_archivo_parquet)
print("Archivo creado")

- - -